In [1]:
from functions import *

In [ ]:
def create_match_download_file(data):
    global match_file
    match_file =  data[['first_name', 'last_name', 'email', 'BUID', 'department', 'current_group']]
    return match_file

def create_download_link(df, title="Download CSV file", filename="data.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    display(HTML(html))

In [2]:
clear_output()
display_widget()

Text(value='example5.xlsx', description='Participants file:', style=DescriptionStyle(description_width='initia…

Button(description='Import', style=ButtonStyle())

Dropdown(description='Matching round:', options=(1, 2, 3, 4, 5, 6, 7, 8, 9), style=DescriptionStyle(descriptio…

Button(description='Create Match', style=ButtonStyle())

Button(description='Download Match', style=ButtonStyle())

Button(description='Complete Match', style=ButtonStyle())

Button(description='Download Data', style=ButtonStyle())

Things to improve:
    - Test what happens when I add someone or remove something
    - Handle it if there's a failure in the matching --> create a flag column for failed pairings or perhaps just throw them into a large group
    - Integrate with and send emails
    - Handle storing the data between pairing rounds